In [4]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
def garbo_distance(x, y):
    """
    Custom distance function for KNN.
    """
    return 0

In [3]:
X = [[1,1],[0,0],[1,1],[1,1],[1,1]]
y = [1, 1, 0, 0, 0]
knn = KNeighborsClassifier(n_neighbors=5, metric=garbo_distance)
knn.fit(X, y)
print(knn.predict([[0, 0], [1, 1]]))  # Should print [0, 1]

[0 0]
